## Relación entre las actividades extracurriculares y el rendimiento académico
**- Hipótesis:**
*Los estudiantes que participan en actividades como leer libros o practicar deportes tienen mejores resultados académicos que aquellos que pasan más tiempo viendo televisión o jugando videojuegos.*


### Selección de datos

Para el dataset `egd10`, disponemos de las siguientes variables a estudiar, divididas en dos grupos según el tipo de actividad:

Hábitos relacionados con **actividades culturales y formativas**:

- `S15A`	Frecuencia uso en casa-Libros de lectura
- `S15B`	Frecuencia uso en casa-Prensa o revistas
- `S15C`	Frecuencia uso en casa-Enciclopedias
- `S20B`	Actividades fuera del colegio-Escuchar música
- `S20C`	Actividades fuera del colegio-Practicar deporte
- `S20D`	Actividades fuera del colegio-Leer libros, cuentos, cómic
- `S20E`	Actividades fuera del colegio-Leer prensa o revistas
- `S20F`	Actividades fuera del colegio-Jugar con amigos
- `S22A`	Internet para-Buscar información para tus estudios
- `S22B`	Internet para-Buscar información sobre deportes
- `S22C`	Internet para-Buscar información sobre juegos
- `S22D`	Internet para-Buscar información sobre música o cine
- `S22E`	Internet para-Buscar información sobre otros temas

Hábitos relacionados con **actividades de ocio digital**:

- `S15D`	Frecuencia uso en casa-Ordenador
- `S15E`	Frecuencia uso en casa-Internet
- `S15F`	Frecuencia uso en casa-Televisión
- `S15G`	Frecuencia uso en casa-Vídeo, DVD
- `S20A`	Actividades fuera del colegio-Ver televisión, vídeos o DVDS
- `S20G`	Actividades fuera del colegio-Hablar por teléfono
- `S20H`	Actividades fuera del colegio-Jugar con videojuegos o con el ordenador
- `S20I`	Actividades fuera del colegio-Participar en chats o messenger
- `S20J`	Actividades fuera del colegio-Usar internet
- `S21A`	Frecuencia utilización ordenador-En casa
- `S21C`	Frecuencia utilización ordenador-En otros lugares
- `S22F`	Internet para-Comunicarme con otras personas

También se han seleccionado las variables relacionadas con el rendimiento académico, como se ha hecho en otras hipótesis.


In [ ]:
import pandas as pd

In [ ]:
egd10 = pd.read_excel("data/EGD10.xlsx")

In [ ]:

posible_features = [
    "S15A", "S15B", "S15C", "S15D", "S15E", "S15F", "S15G",
    "S20A", "S20B", "S20C", "S20D", "S20E", "S20F", "S20G", "S20H", "S20I", "S20J",
    "S21A", "S21B", "S21C",
    "S22A", "S22B", "S22C", "S22D", "S22E", "S22F"
]

targets = list(map(lambda x: x.upper(), [
    "PV1ling",
    "PV2ling",
    "PV3ling",
    "PV4ling",
    "PV5ling",
    "PV1mat",
    "PV2mat",
    "PV3mat",
    "PV4mat",
    "PV5mat",
    "PV1mfis",
    "PV2mfis",
    "PV3mfis",
    "PV4mfis",
    "PV5mfis",
    "PV1syc",
    "PV2syc",
    "PV3syc",
    "PV4syc",
    "PV5syc",
]))



In [ ]:
egd10 = egd10[targets + posible_features]
egd10.dropna(inplace=True)


### Tratamiento y limpieza de datos


La transformación de las variables requerirá de la agrupación de las mismas, para ello tendremos que realizar previamente una unificación de las unidades de medida. Adicionalmente, para que esto sea posible, tendremos que trater primero los valores nulos de las variables que vayamos a tener en cuenta.

#### Tratamiento de valores nulos

En relación con las variables seleccionadas, será necesario realizar un tratamiento específico de los valores por defecto. En todas estas variables, el valor 9 se utiliza como marcador por defecto, mientras que el valor 1 se emplea para indicar ausencia o no uso.

Por esta razón, procederemos a descartar aquellas filas en las que el valor de la variable sea 9, ya que no aportan información relevante para el análisis.


In [ ]:
# Eliminar valores nulos
egd10 = egd10[(egd10[posible_features] != 9).all(axis=1)]

Para el caso de las variables de `target`, procederemos de igual manera que con las anteriores hipótesis.

#### Tratamiento de las variables

En primer lugar, agrupamos las variables de `target`:

In [ ]:
egd10.loc[:, "Nota_Media"] = egd10[targets].mean(axis=1)
egd10.drop(targets, axis=1, inplace=True)
egd10.loc[:, "Nota_Media"] = egd10["Nota_Media"].apply(lambda x: x / 100)

Las variables `S15x` los valores 5 se asume que tienen el mismo valor que 1 ( No tener es igual que no dedicarle tiempo )

In [ ]:
# Cambiar los valores 5 por 1 en las columnas que comienzan con "S15"
for col in posible_features:
    if col.startswith("S15"):
        egd10[col] = egd10[col].replace(5, 1)

Ahora vamos a analizar las variables para seleccionar las que mas influe `Nota_Media`

In [ ]:
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt

# Seleccionar las características para calcular la matriz de correlación
df_feat = egd10[["Nota_Media"] + posible_features]

# Calcular la matriz de correlación
corr = df_feat.corr()

# Generar una máscara para el triángulo superior
mask = np.triu(np.ones_like(corr, dtype=bool))

# Configurar la figura de matplotlib
plt.figure(figsize=(11, 9))

# Generar un colormap divergente personalizado
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Dibujar el heatmap con la máscara y la relación de aspecto correcta
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.show()

Como no hay una suficiente relaccion vamos a ver cuales influyen mas por proximidad, y descartar el resto

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist

# Calcular la matriz de distancias
dist_matrix = pdist(corr)

# Realizar el enlace jerárquico
Z = linkage(dist_matrix, method='ward')

# Dibujar el dendrograma
plt.figure(figsize=(15, 10))
dendrogram(Z, labels=corr.columns, leaf_rotation=90, leaf_font_size=10)
plt.title('Dendrograma de las variables')
plt.xlabel('Variables')
plt.ylabel('Distancia')
plt.show()

Para el caso de las variables seleccionadas, observamos en primer lugar que todas son categóricas ordinales.

Para las variables 15[A-E] 20[A-J], la medida es tiempo en horas, aunque no queda claro si es al día o a la semana.
El máximo es de 2h, por lo que se ha decidido considerar que es al día.

En cuanto a 21[A-C] y 22[A-F], la medida es la frecuencia de uso.


Para poder unificar las variables, haremos la siguiente conversión a "tiempo dedicado", medido en horas:

Para la variable de tiempo:

- Nada de tiempo → 0 horas
- Menos de 30 minutos → 0.25 horas
- Entre 30 y 60 minutos → 0.75 horas
- Entre 1 y 2 horas → 1.5 horas
- Más de 2 horas → 2.5 horas

Para la variable de frecuencia:

- Nunca o casi nunca → 0 horas (se puede tratar como "0 tiempo dedicado").
- Una o dos veces al mes → 0.2 horas (aproximadamente equivalente a dedicar menos de 30 minutos en promedio semanal).
- Una o dos veces a la semana → 0.65 horas (aproximadamente 30-60 minutos a la semana).
- Todos o casi todos los días → 1.25 horas (equivalente a dedicar una hora y pico diaria en promedio).

Lo que haremos es mapear la conversión a cada una de las variables y después haremos un total agrupando en "actividades de ocio digital" o "actividades culturales y formativas":

In [ ]:
# Mapeos para las variables de tiempo y frecuencia
# Los valores representan horas estimadas para cada categoría
map_tiempo = {1: 0, 2: 0.25, 3: 0.75, 4: 1.5, 5: 2.5}
map_frecuencia = {1: 0, 2: 0.2, 3: 0.65, 4: 1.25}



# Listas de características según tipo de actividad
# Variables relacionadas con actividades digitales
tiempo_features_digital = ["S15D", "S15E", "S20I", "S20J"]
frecuencia_features_digital = ["S21A","S22D", "S22F", "S22E"]

# Variables relacionadas con actividades culturales
tiempo_features_cultural = ["S15C", "S15A", "S20D"]

# Conversión de variables digitales
# Mapear las categorías numéricas a horas estimadas para cada variable de tiempo y frecuencia digital
for feature in tiempo_features_digital:
    egd10[feature + "_horas_digital"] = egd10[feature].map(map_tiempo)

for feature in frecuencia_features_digital:
    egd10[feature + "_horas_digital"] = egd10[feature].map(map_frecuencia)

# Sumar todas las horas digitales en una nueva columna unificada
# Combina tiempo y frecuencia para calcular un total de horas invertidas en actividades digitales
egd10["Total_Horas_Digital"] = egd10[[feature + "_horas_digital" for feature in tiempo_features_digital + frecuencia_features_digital]].sum(axis=1)

# Conversión de variables culturales
# Mapear las categorías numéricas a horas estimadas para cada variable de tiempo y frecuencia cultural
# Sumar todas las horas culturales en una nueva columna unificada
# Combina tiempo y frecuencia para calcular un total de horas invertidas en actividades culturales
egd10['Total_Horas_Cultural'] = egd10[["S15C", "S15A", "S20D"]].applymap(map_tiempo.get).sum(axis=1) + egd10[["S21A", "S22D", "S22F", "S22E"]].applymap(map_frecuencia.get).sum(axis=1)



In [ ]:
egd10.head()

### Características de la tarjeta de datos


El nuevo dataset generado, `tarjeta_H4`, representa una versión simplificada y procesada de los datos originales, obtenida a través de la transformación y selección de las características más relevantes para la hipotésis que nos atañe en este caso.
Las transformaciones empleada en las características de importancia seleccionadas ha sido principalmente la reducción de la dimensionalidad mediante la suma, y la conversión de tipos de datos por medio de mapeo.

In [ ]:
# Filtrar las columnas deseadas
egd10_h4 = egd10[["Total_Horas_Cultural", "Total_Horas_Digital", "Nota_Media"]]

# Guardar el nuevo DataFrame en un archivo CSV
egd10_h4.to_csv("data/tarjeta_H4.csv", index=False, float_format="%.2f")

Las variables finales del dataset son:

- `Total_Horas_Cultural`: total de horas diarias que el estudiante dedica a ocio del tipo *actividades culturales y formativas*
- `Total_Horas_Digital`: total de horas diarias que el estudiante dedica a ocio del tipo *actividades de ocio digital*
- `Nota_Media`: nota media del estudiante

Sin valores nulos ni valores por defecto.

### Líneas de Trabajo


Para validar la hipótesis de que el ocio de los estudiantes influye en su rendimiento académico, se podría realizar un análisis de correlación entre las variables que reflejan el tipo de ocio frecuentado y su tiempo de dedicación y sus calificaciones. Se podrán técnicas de clustering para identificar agrupaciones y patrones entre los datos.

Este análisis permitirá explorar la posible conexión entre ambas variables, cuantificando tanto su relevancia estadística como la intensidad de su impacto.

In [ ]:
# Definir el umbral de diferencia
umbral = 1.0  # Ajusta este valor según la sensibilidad deseada

# Añadir un valor mínimo a 'Horas_Digital ya que trabajamos con un rango muy amplio, los valores bajos son despreciables para nuestro análisis'
egd10['Total_Horas_Digital'] = egd10['Total_Horas_Digital'].apply(lambda x: max(x, 5))

# Clasificar en tres categorías
egd10['Mayor_Horas'] = egd10.apply(
    lambda row: (
        'Cultural' if row['Total_Horas_Cultural'] > row['Total_Horas_Digital'] + umbral else
        'Digital' if row['Total_Horas_Digital'] > row['Total_Horas_Cultural'] + umbral else
        'Igualado'
    ),
    axis=1
)
# Mapear las categorías de 'Mayor_Horas' a valores numéricos
egd10['Mayor_Horas_Bin'] = egd10['Mayor_Horas'].apply(
    lambda x: 1 if x == 'Cultural' else -1 if x == 'Digital' else 0
)

# Verificar las frecuencias de los valores binarios
print(egd10['Mayor_Horas_Bin'].value_counts())


# Verificar las frecuencias de cada categoría
print(egd10['Mayor_Horas'].value_counts())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Gráfico KDE para `Nota_Media` según `Mayor_Horas_Bin`
sns.kdeplot(egd10["Nota_Media"][egd10.Mayor_Horas_Bin == 1], label='Cultural', fill=True)
sns.kdeplot(egd10["Nota_Media"][egd10.Mayor_Horas_Bin == -1], label='Digital', fill=True)

# Calcular las medias
media_cultural = egd10["Nota_Media"][egd10.Mayor_Horas_Bin == 1].mean()
media_digital = egd10["Nota_Media"][egd10.Mayor_Horas_Bin == -1].mean()

# Marcar líneas de la media
plt.axvline(media_cultural, color='blue', linestyle='--', label='Media Cultural')
plt.axvline(media_digital, color='orange', linestyle='--', label='Media Digital')

# Personalización de etiquetas
plt.xlabel('Nota Media')
plt.title('Distribución de Nota Media por Categoría de Mayor Horas')
plt.legend()
plt.show()



## Justificación del Modelado Utilizado

Se utilizaron técnicas de análisis de correlación y clustering jerárquico debido a la naturaleza de las variables y la hipótesis planteada. Estas técnicas permiten identificar relaciones y patrones en los datos, así como agrupar variables similares, lo cual es esencial para entender la influencia de las actividades extracurriculares en el rendimiento académico.

## Resultados Obtenidos e Interpretación

- **Análisis de Correlación**:
  - No se encontró una correlación significativa entre las variables de actividades extracurriculares y la nota media.

- **Clustering Jerárquico**:
  - Se identificaron agrupaciones de variables que muestran similitudes en sus patrones de dedicación de tiempo.

- **Distribución de Nota Media**:
  - Los estudiantes que dedican más tiempo a actividades culturales tienden a tener una nota media ligeramente superior en comparación con aquellos que dedican más tiempo a actividades digitales.
  - La media de la nota para los estudiantes que dedican más tiempo a actividades culturales es de 5.58, mientras que para aquellos que dedican más tiempo a actividades digitales es de 5.03.

Estos resultados sugieren que, aunque no hay una correlación fuerte, existe una tendencia que apoya la hipótesis de que las actividades culturales pueden tener un impacto positivo en el rendimiento académico. Sin embargo, se recomienda realizar análisis adicionales para confirmar estos hallazgos y explorar otras posibles variables influyentes.